In [1]:
#This section has been modified based on the open-source code provided by PeSTO. For specific usage details, please refer to https://github.com/SonyCSLParis/pesto
import os
import sys
import h5py
import json
import numpy as np
import torch as pt
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob

from src.dataset import StructuresDataset, collate_batch_features, select_by_sid, select_by_interface_types
from src.data_encoding import encode_structure, encode_features, extract_topology, categ_to_resnames, resname_to_categ
from src.structure import data_to_structure, encode_bfactor, concatenate_chains, split_by_chain
from src.structure_io import save_pdb, read_pdb
from src.scoring import bc_scoring, bc_score_names

# data parameters
data_path = "/your/PDB/file/path"
# model parameters
# R3
#save_path = "model/save/i_v3_0_2021-05-27_14-27"  # 89
#save_path = "model/save/i_v3_1_2021-05-28_12-40"  # 90
# R4
#save_path = "model/save/i_v4_0_2021-09-07_11-20"  # 89
save_path = "model/save/i_v4_1_2021-09-07_11-21"  # 91
# save_path = "/mnt/disk1/guoxiaokun/isoform/PeSTO/PeSTo-main/examples/isoform/save"
# select saved model
model_filepath = os.path.join(save_path, 'model_ckpt.pt')
#model_filepath = os.path.join(save_path, 'model.pt')

In [4]:
# add module to path
if save_path not in sys.path:
    sys.path.insert(0, save_path)
    
# load functions
from config import config_model, config_data
from data_handler import Dataset
from model import Model

device = pt.device("cpu")
model = Model(config_model)
model.load_state_dict(pt.load(model_filepath, map_location=pt.device("cpu")))
model = model.eval().to(device)
# find pdb files and ignore already predicted oins
pdb_filepaths = glob(os.path.join(data_path, "*.pdb"), recursive=True)
pdb_filepaths = [fp for fp in pdb_filepaths if "_i" not in fp]
# create dataset loader with preprocessing
dataset = StructuresDataset(pdb_filepaths, with_preprocessing=True)

# debug print
print(len(dataset))

In [ ]:
# run model on all subunits
protein_dict = {}
with pt.no_grad():
    for subunits, filepath in tqdm(dataset):
        # concatenate all chains together
        structure = concatenate_chains(subunits)
        # encode structure and features
        X, M = encode_structure(structure)
        #q = pt.cat(encode_features(structure), dim=1)
        q = encode_features(structure)[0]
        # extract topology
        ids_topk, _, _, _, _ = extract_topology(X, 64)
        X, ids_topk, q, M = collate_batch_features([[X, ids_topk, q, M]])
        # run model
        z = model(X.to(device), ids_topk.to(device), q.to(device), M.float().to(device))
        element = filepath[:-4].split('/')[-1]
        proval = pt.sigmoid(z[:,0]).tolist()
        proval_str = "\t".join(map(str, proval))
        protein_dict[element] = proval_str

In [13]:
#The output file will be used as input for the ESM2 GCN model.
with open("/your/path/geometirc.allfeature.txt", "w") as f:
    for key, value in protein_dict.items():
        f.write(f"{key}\t{value}\n")